<h3 align="center">Codebasics ML Course: Grid Search CV</h3>

We will generate a synthetic dataset

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_features=10, 
    n_samples=1000, 
    n_informative=8,
    n_redundant=2,
    n_repeated=0,
    n_classes=2, 
    random_state=42
)

### Method 1: Evaluate the model using train, test split and tune parameters by trial and error

In [22]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model = DecisionTreeClassifier(criterion="entropy", max_depth=10) # criteria: "gini" or "entropy", max_depth=5 or 10
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.85      0.77      0.81       130
           1       0.77      0.85      0.81       120

    accuracy                           0.81       250
   macro avg       0.81      0.81      0.81       250
weighted avg       0.81      0.81      0.81       250



### Method 2: Use K Fold Cross Validation

In [24]:
from sklearn.model_selection import cross_val_score

cross_val_score(DecisionTreeClassifier(criterion="gini", max_depth=5), X, y, cv=5)

array([0.775, 0.79 , 0.735, 0.79 , 0.775])

In [25]:
cross_val_score(DecisionTreeClassifier(criterion="entropy", max_depth=5), X, y, cv=5)

array([0.765, 0.785, 0.755, 0.815, 0.78 ])

In [27]:
criterion = ["gini", "entropy"]
max_depth = [5, 10, 15]

avg_scores = {}

for c in criterion:
    for d in max_depth:
        clf = DecisionTreeClassifier(criterion=c, max_depth=d)
        score_list = cross_val_score(clf, X, y, cv=5)
        avg_scores[c + "_" + str(d)] = np.average(score_list)
        
avg_scores

{'gini_5': 0.783,
 'gini_10': 0.787,
 'gini_15': 0.788,
 'entropy_5': 0.7769999999999999,
 'entropy_10': 0.792,
 'entropy_15': 0.813}

### Method 3: Use GridSearchCV

In [28]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(
    DecisionTreeClassifier(),
    {'criterion': ["gini", "entropy"],'max_depth': [5, 10, 15]},
    cv=5,
    return_train_score=False
)
clf.fit(X, y)
clf.cv_results_

{'mean_fit_time': array([0.01029077, 0.01576133, 0.02004147, 0.00938282, 0.01401248,
        0.01252565]),
 'std_fit_time': array([0.0067583 , 0.00611863, 0.00630879, 0.00766104, 0.00425623,
        0.00871227]),
 'mean_score_time': array([0.00566778, 0.        , 0.00040321, 0.00311775, 0.00020132,
        0.00313172]),
 'std_score_time': array([0.00611845, 0.        , 0.00049384, 0.0062355 , 0.00040264,
        0.00626345]),
 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                    'entropy'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[5, 10, 15, 5, 10, 15],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'criterion': 'gini', 'max_depth': 5},
  {'criterion': 'gini', 'max_depth': 10},
  {'criterion': 'gini', 'max_depth': 15},
  {'criterion': 'entropy',

In [30]:
df = pd.DataFrame(clf.cv_results_)
df.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010291,0.006758,0.005668,0.006118,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.780,0.815,0.745,0.795,0.770,0.781,0.023537,5
1,0.015761,0.006119,0.000000,0.000000,gini,10,"{'criterion': 'gini', 'max_depth': 10}",0.790,0.730,0.815,0.800,0.810,0.789,0.030725,3
2,0.020041,0.006309,0.000403,0.000494,gini,15,"{'criterion': 'gini', 'max_depth': 15}",0.805,0.725,0.800,0.800,0.805,0.787,0.031081,4


In [31]:
df[["param_criterion", "param_max_depth", "mean_test_score"]]

,param_criterion,param_max_depth,mean_test_score
0,gini,5,0.781
1,gini,10,0.789
2,gini,15,0.787
3,entropy,5,0.777
4,entropy,10,0.792
5,entropy,15,0.813


In [32]:
clf.best_params_

{'criterion': 'entropy', 'max_depth': 15}

In [33]:
model = clf.best_estimator_
model

DecisionTreeClassifier(criterion='entropy', max_depth=15)

### Now let's try different models with different parameters

In [36]:
from sklearn import svm

model_params = {
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params' : {
            'criterion': ["gini", "entropy"],
            'max_depth': [5, 10, 15]
        }  
    },
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']            
        }
    }
}

scores = []

for key, val in model_params.items():
    clf = GridSearchCV(
        val['model'],
        val['params'],
        cv=5,
        return_train_score=False
    )
    clf.fit(X, y)
    scores.append({
        'model': key,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
scores

[{'model': 'decision_tree',
  'best_score': 0.812,
  'best_params': {'criterion': 'entropy', 'max_depth': 15}},
 {'model': 'svm',
  'best_score': 0.9260000000000002,
  'best_params': {'C': 1, 'kernel': 'rbf'}}]

In [37]:
df = pd.DataFrame(scores, columns=["model", "best_score", "best_params"])
df

,model,best_score,best_params
0,decision_tree,0.812,"{'criterion': 'entropy', 'max_depth': 15}"
1,svm,0.926,"{'C': 1, 'kernel': 'rbf'}"
